In [ ]:
import pygame
import random
import os
import shutil

# Initialize Pygame and Mixer
pygame.init()
pygame.mixer.init()

# Get the current working directory
current_dir = os.getcwd()

# Define folder paths for music and images
music_folder = os.path.join(current_dir, "music")
images_folder = os.path.join(current_dir, "images")

# Create folders if they don't exist
if not os.path.exists(music_folder):
    os.makedirs(music_folder)
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

# Define source paths for music and image
source_music = r'C:\Users\katya\Downloads\i_te_gidiyorum.mp3'  # Replace with your music file path
source_image = r'C:\Users\katya\Downloads\snk.png'  # New background image path

# Define destination paths
dest_music = os.path.join(music_folder, "back.mp3")
dest_image = os.path.join(images_folder, "snake.png")

# Copy the files to the working directory
shutil.copy(source_music, dest_music)
shutil.copy(source_image, dest_image)

# Define file paths for the game
music_path = os.path.join(music_folder, "back.mp3")  # Music file in 'music' folder
bgimg_path = os.path.join(images_folder, "snake.png")  # Background image in 'images' folder

# Check if the music file exists
if not os.path.isfile(music_path):
    raise FileNotFoundError(f"Music file '{music_path}' not found. Please place the music file in 'music' folder.")

# Load and play background music
pygame.mixer.music.load(music_path)
pygame.mixer.music.play(-1)  # Loop the music indefinitely

# Set the window size and caption
screen_width = 1200
screen_height = 600
gameWindow = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("First Game of DK")

# Load and scale the background image
if not os.path.isfile(bgimg_path):
    raise FileNotFoundError(f"Background image '{bgimg_path}' not found. Please place the image in 'images' folder.")

bgimg = pygame.image.load(bgimg_path)
bgimg = pygame.transform.scale(bgimg, (screen_width, screen_height)).convert_alpha()

# Define colors
white = (255, 255, 255)
red = (255, 0, 0)
black = (0, 0, 0)

# Font for text rendering
font = pygame.font.SysFont(None, 55)

def text_screen(text, color, x, y):
    screen_text = font.render(text, True, color)
    gameWindow.blit(screen_text, [x, y])

def plot_snake(gameWindow, color, snk_list, snake_size):
    for x, y in snk_list:
        pygame.draw.rect(gameWindow, color, [x, y, snake_size, snake_size])

def read_highscore():
    try:
        with open("highscore.txt", "r") as f:
            return int(f.read())
    except FileNotFoundError:
        return 0
    except ValueError:
        return 0

def write_highscore(score):
    with open("highscore.txt", "w") as f:
        f.write(str(score))

def welcome():
    exit_game = False
    clock = pygame.time.Clock()
    while not exit_game:
        gameWindow.blit(bgimg, (0, 0))
        text_screen("Welcome to Snake Game", black, 600, 100)  # Adjusted x-coordinate for more right shift
        text_screen("Press Space to Play", black, 750, 150)    # Further moved to the right

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                exit_game = True  # Corrected: Close the game window properly
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    gameloop()  # Start the game when space is pressed
                if event.key == pygame.K_q:
                    exit_game = True  # Quit when 'q' is pressed

        pygame.display.update()
        clock.tick(60)
    pygame.quit()  # Quit pygame correctly when the window is closed
    quit()         # Terminate the program completely

def gameloop():
    highscore = read_highscore()
    exit_game = False
    game_over = False

    # Game loop variables
    snake_x = 45
    snake_y = 55
    velocity_x = 0
    velocity_y = 0
    snk_list = []
    snk_length = 1
    food_x = random.randint(20, int(screen_width / 1.5))
    food_y = random.randint(20, int(screen_height / 1.5))

    score = 0
    init_velocity = 5
    snake_size = 30
    fps = 60
    clock = pygame.time.Clock()

    while not exit_game:
        if game_over:
            write_highscore(highscore)
            gameWindow.blit(bgimg, (0, 0))
            text_screen("Game Over! Press Enter to Restart or Q to Quit", red, 100, 250)
            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    exit_game = True
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RETURN:
                        gameloop()  # Restart the game
                    if event.key == pygame.K_q:
                        exit_game = True

        else:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    exit_game = True
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RIGHT:
                        velocity_x = init_velocity
                        velocity_y = 0
                    elif event.key == pygame.K_LEFT:
                        velocity_x = -init_velocity
                        velocity_y = 0
                    elif event.key == pygame.K_UP:
                        velocity_y = -init_velocity
                        velocity_x = 0
                    elif event.key == pygame.K_DOWN:
                        velocity_y = init_velocity
                        velocity_x = 0

            snake_x += velocity_x
            snake_y += velocity_y

            if abs(snake_x - food_x) < 6 and abs(snake_y - food_y) < 10:
                score += 10
                food_x = random.randint(20, int(screen_width / 1.5))
                food_y = random.randint(20, int(screen_height / 1.5))
                snk_length += 5
                if score > highscore:
                    highscore = score

            if snake_x < 0 or snake_x > screen_width or snake_y < 0 or snake_y > screen_height:
                game_over = True

            head = [snake_x, snake_y]
            snk_list.append(head)

            if len(snk_list) > snk_length:
                del snk_list[0]

            if head in snk_list[:-1]:
                game_over = True

            gameWindow.blit(bgimg, (0, 0))
            text_screen("Score: " + str(score) + " Highscore: " + str(highscore), red, 5, 5)
            plot_snake(gameWindow, black, snk_list, snake_size)
            pygame.draw.rect(gameWindow, red, [food_x, food_y, snake_size, snake_size])

        pygame.display.update()
        clock.tick(fps)

    pygame.quit()  # Quit pygame correctly when the game is over
    quit()         # Terminate the program completely

# Start the game
welcome()
